# Accuracy Measure for FASHION MNIST via CNN 


In [1]:
using Flux
using Flux: crossentropy, throttle
using Flux: @epochs
using MLDatasets
using Base.Iterators: partition
using Flux: onehotbatch
using Random
train_x, train_y = FashionMNIST.traindata();
test_x,  test_y  = FashionMNIST.testdata();

Model for CNN with filter size (3,3), max pooling size (2,2) and using relu activation functions for intermediate 
layers, softmax for the outer layer. For the preprocessing convolutional layers we use output dimensions in successions; 32,64,128. 

We further improve the accuracy to 86.75% amd with more hyper parameter tuning, it is quite likely that accuracy could go higher than 90%

In [2]:
using Flux: onecold
using Statistics
function CNN(train_x, train_y, test_input, test_output, iter = 10, call = 1, train_examples = 2000, batch_size = 50)

    model = Chain(
        Conv((3,3), 1=>32, relu),
        x -> maxpool(x, (2,2)),
        #x -> (Dropout(0.25))(x),
        Conv((3,3), 32=>64, relu),
        x -> maxpool(x, (2,2)),
        Conv((3,3), 64=>128, relu),
        x -> maxpool(x, (2,2)),
        x -> reshape(x, :, size(x, 4)),
        Dense(128, 10),
        softmax)

    loss(x, y) = crossentropy(model(x), y)
    opt = ADAM(params(model))
    trainIndex = randperm(size(train_x)[end])[1:2000]
    trainData = []
    for batch in partition(1:train_examples, batch_size)
        trainXFloat = Float64.(train_x[:, :, batch])
        trainXReshaped = reshape(trainXFloat, (28, 28, 1, length(batch)))
        trainY = onehotbatch(train_y[batch], 0:9)
        push!(trainData, (trainXReshaped, trainY))
    end
test_foracc_input = deepcopy(test_input)
test_foracc_output = deepcopy(test_output)
test_input = reshape(test_input, (size(test_input)[1],size(test_input)[2],1,size(test_input)[3]))    
test_output = onehotbatch(test_output, 0:9)
test_input = convert(Array{Float64}, test_input);

println(size(test_input))
println(size(test_output))
println(typeof(test_input))
println(typeof(test_output))
    if call == 0
        callback() = @show(loss(test_input, test_output))    
        @time @epochs iter Flux.train!(loss, trainData, opt, cb = throttle(callback, 5))
    else
        
        @time @epochs iter Flux.train!(loss, trainData, opt)
    end
predicted = model(reshape(Float64.(test_foracc_input), (28, 28, 1, 10000)))
accuracy = mean(onecold(predicted).-1 .== test_foracc_output)
println(accuracy*100)   
return accuracy    
end
acc = CNN(train_x,train_y, test_x, test_y, 50,1,60000);

(28, 28, 1, 10000)
(10, 10000)
Array{Float64,4}
Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}


┌ Info: Epoch 1
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 2
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 3
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 4
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 5
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 6
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 7
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 8
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 9
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 10
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: Epoch 11
└ @ Main /home/ashvin/.julia/packages/Flux/rcN9D/src/optimise/train.jl:93
┌ Info: 

11895.757063 seconds (21.54 G allocations: 6.462 TiB, 39.07% gc time)
86.75
